# Spreedly.ai - API + LLM Demo

This notebook is "inspired" by 🦜⛓️'s OpenAPI [agents](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/openapi.html) and [chains](https://github.com/hwchase17/langchain/blob/master/docs/modules/chains/examples/openapi.ipynb) examples.

## Requirements

- Ensure you have selected the Jupyter-provided `venv`
- Remember to run `python3 -m venv venv`
- Create a `.env` with the following format:
```
SPREEDLY_OPEN_API_SPEC_YAML_LOC=path/to/spec.yaml
SPREEDLY_ACCESS_TOKEN=<base64 encoded user:pass>
OPENAI_API_KEY=<your OPEN AI API KEY>
```

## Load Spreedly API Spec

In [1]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

yaml_location =  os.getenv('SPREEDLY_OPEN_API_SPEC_YAML_LOC')
with open(yaml_location) as f:
    raw_spreedly_openai_api_spec = yaml.load(f, Loader=yaml.Loader)

spreedly_openai_api_spec = reduce_openapi_spec(raw_spreedly_openai_api_spec)

In [2]:
from langchain.requests import RequestsWrapper

access_token = os.getenv('SPREEDLY_ACCESS_TOKEN')
headers = {
        'Authorization': f'Basic {access_token}'
    }
requests_wrapper = RequestsWrapper(headers=headers)

Kick the tires, can we instrospect the API?

In [3]:
import json

endpoints = [
    (route, operation)
    for route, operations in raw_spreedly_openai_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]

display(f"There are {len(endpoints)} endpoints available")

display(f"""Server Base URL: {raw_spreedly_openai_api_spec["servers"][0]["url"]}""")

'There are 33 endpoints available'

'Server Base URL: https://core.spreedly.com/v1'

Prepare our objects

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits.openapi import planner


# relies on OPENAI_API_KEY from .env file.
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)

Create Agent

In [5]:
spreedly_agent = planner.create_openapi_agent(spreedly_openai_api_spec, requests_wrapper, llm, verbose=True)

## List Supported Gateways

In [6]:
user_query = "List all gateways supported by Spreedly"

spreedly_agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to list all gateways supported by Spreedly
Observation: 1. GET /gateways_options.json to retrieve a list of all gateways and their properties supported by Spreedly.
Thought:I have the plan to get the list of supported gateways. Now, I will execute the API calls.
Action: api_controller
Action Input: 1. GET /gateways_options.json to retrieve a list of all gateways and their properties supported by Spreedly.

> Entering new AgentExecutor chain...
I will start by making a GET request to the /gateways_options.json endpoint to retrieve the list of all gateways and their properties supported by Spreedly.

Action: requests_get
Action Input: {"url": "https://core.spreedly.com/v1/gateways_options.json", "params": {}, "output_instructions": "Extract the list of gateways and their properties"}

Observation: [{"gateway_type": "acapture", "auth_modes": ["default"], "characteristics": ["purcha

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


I have executed the plan and retrieved the list of supported gateways. I can now provide the user with the information they requested.
Final Answer: Here is a list of gateways supported by Spreedly: [{"gateway_type": "acapture", "auth_modes": ["default"], "characteristics": ["purchase", "authorize", "capture", "credit", "void", "verify"], "payment_methods": ["credit_card"], "gateway_settings": [], "gateway_specific_fields": ["disable_3d_secure"], "supported_countries": ["AD", "AI", "AT", "BE", "BG", "CA", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HK", "HU", "IS", "IT", "LV", "LI", "LT", "LU", "MT", "MC", "NL", "PL", "PT", "SM", "SK", "SI", "ES", "SE", "CH", "TR", "GB", "US"], "supported_cardtypes": ["visa", "master", "american_express", "diners_club", "discover", "jcb", "maestro", "dankort"], "regions": ["asia_pacific", "europe", "north_america"]}]

> Finished chain.


'Here is a list of gateways supported by Spreedly: [{"gateway_type": "acapture", "auth_modes": ["default"], "characteristics": ["purchase", "authorize", "capture", "credit", "void", "verify"], "payment_methods": ["credit_card"], "gateway_settings": [], "gateway_specific_fields": ["disable_3d_secure"], "supported_countries": ["AD", "AI", "AT", "BE", "BG", "CA", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HK", "HU", "IS", "IT", "LV", "LI", "LT", "LU", "MT", "MC", "NL", "PL", "PT", "SM", "SK", "SI", "ES", "SE", "CH", "TR", "GB", "US"], "supported_cardtypes": ["visa", "master", "american_express", "diners_club", "discover", "jcb", "maestro", "dankort"], "regions": ["asia_pacific", "europe", "north_america"]}]'

## Create a gateway (sandbox mode)

 Some glue code first

In [7]:
%%script false --no-raise-error
## Currently disabled.

import os

adyen_creds = f"""{{
    "ADYEN_USERNAME": {os.getenv('ADYEN_USERNAME')},
    "ADYEN_PASSWORD": {os.getenv('ADYEN_PASSWORD')},
    "ADYEN_MERCHANT_ACCOUNT": {os.getenv('ADYEN_MERCHANT_ACCOUNT')}
}}
"""


Send the query

In [8]:
%%script false --no-raise-error
## Currently disabled.

spreedly_agent = planner.create_openapi_agent(spreedly_openai_api_spec, requests_wrapper, llm, verbose=True)

user_query ="Create a new adyen sandbox gateway using these gateway credentials: {adyen_creds}"

spreedly_agent.run(user_query)